In [30]:
# 加载数据集
dataset_path = "C:/Users/11730/Downloads/祝福.txt"

with open(dataset_path, "r", encoding="utf-8") as f:
    data = f.read()

# 打印数据的一部分，检查是否正确加载
print(data[:500])  # 打印前500个字符

旧历的年底毕竟最像年底，村镇上不必说，就在天空中也显出将到新年的气象来。灰白色的沉重的晚云中间时时发出闪光，接着一声钝响，是送灶的爆竹；近处燃放的可就更强烈了，震耳的大音还没有息，空气里已经散满了幽微的火药香。我是正在这一夜回到我的故乡鲁镇的。虽说故乡，然而已没有家，所以只得暂寓在鲁四老爷的宅子里。他是我的本家，比我长一辈，应该称之曰“四叔”，是一个讲理学的老监生。他比先前并没有什么大改变，单是老了些，但也还末留胡子，一见面是寒暄，寒暄之后说我“胖了”，说我“胖了”之后即大骂其新党。但我知道，这并非借题在骂我：因为他所骂的还是康有为。但是，谈话是总不投机的了，于是不多久，我便一个人剩在书房里。 
　　第二天我起得很迟，午饭之后，出去看了几个本家和朋友；第三天也照样。他们也都没有什么大改变，单是老了些；家中却一律忙，都在准备着“祝福”。这是鲁镇年终的大典，致敬尽礼，迎接福神，拜求来年一年中的好运气的。杀鸡，宰鹅，买猪肉，用心细细的洗，女人的臂膊都在水里浸得通红，有的还带着绞丝银镯子。煮熟之后，横七竖八的插些筷子在这类东西上，可就称为“福礼”了，五更天陈列起来，并且点上香烛，恭请福神们来享


In [31]:
import numpy as np

# 构建字符到数字的映射
chars = sorted(list(set(data)))  # 获取所有字符
vocab_size = len(chars)
char_to_index = {ch: i for i, ch in enumerate(chars)}
index_to_char = {i: ch for i, ch in enumerate(chars)}

# 将文本数据转换为数字列表
data_as_numbers = [char_to_index[c] for c in data]

# 分割成训练数据和测试数据（80%训练，20%测试）
train_data = data_as_numbers[:int(0.8 * len(data_as_numbers))]
test_data = data_as_numbers[int(0.8 * len(data_as_numbers)):]

# 转换成适合训练的小批次
block_size = 128  # 每个训练块的大小
X = []
Y = []
for i in range(0, len(train_data) - block_size, block_size):
    X.append(train_data[i:i + block_size])
    Y.append(train_data[i + 1:i + 1 + block_size])

X = np.array(X)
Y = np.array(Y)

# 查看处理后的数据
print(X.shape, Y.shape)

(60, 128) (60, 128)


In [32]:
import torch
import torch.nn as nn
import torch.optim as optim

class NanoGPT(nn.Module):
    def __init__(self, vocab_size, block_size, n_layer=8, n_head=8, n_embd=256):
        super(NanoGPT, self).__init__()
        self.block_size = block_size
        self.token_emb = nn.Embedding(vocab_size, n_embd)
        self.pos_emb = nn.Embedding(block_size, n_embd)
        self.transformer_blocks = nn.ModuleList([
            nn.TransformerEncoderLayer(n_embd, n_head) for _ in range(n_layer)
        ])
        self.ln_f = nn.LayerNorm(n_embd)
        self.head = nn.Linear(n_embd, vocab_size)

    def forward(self, x):
        token_embeddings = self.token_emb(x)  # [batch_size, block_size, n_embd]
        pos_embeddings = self.pos_emb(torch.arange(self.block_size, device=x.device))  # [block_size, n_embd]
        x = token_embeddings + pos_embeddings
        for block in self.transformer_blocks:
            x = block(x)
        x = self.ln_f(x)
        logits = self.head(x)  # [batch_size, block_size, vocab_size]
        return logits

# 设置超参数
vocab_size = len(chars)
block_size = 128
n_layer = 8
n_head = 8
n_embd = 256

# 创建模型
model = NanoGPT(vocab_size, block_size, n_layer, n_head, n_embd)

# 设置优化器和损失函数
optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

# 训练模型
epochs = 10
batch_size = 64

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for i in range(0, len(X), batch_size):
        inputs = torch.tensor(X[i:i + batch_size])
        targets = torch.tensor(Y[i:i + batch_size])

        optimizer.zero_grad()
        logits = model(inputs)
        loss = loss_fn(logits.view(-1, vocab_size), targets.view(-1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(X)}")

Epoch 1/10, Loss: 0.12034445603688558
Epoch 2/10, Loss: 0.10804674625396729
Epoch 3/10, Loss: 0.10529685815175374
Epoch 4/10, Loss: 0.10181323687235515
Epoch 5/10, Loss: 0.09975880781809489
Epoch 6/10, Loss: 0.09828195571899415
Epoch 7/10, Loss: 0.0969465732574463
Epoch 8/10, Loss: 0.09598671595255534
Epoch 9/10, Loss: 0.09543030261993408
Epoch 10/10, Loss: 0.0951140562693278


In [39]:
def generate_text(model, start_text, length=100):
    model.eval()

    # 确保输入文本的长度符合模型要求（假设 block_size=128）
    block_size = model.block_size  # 模型的 block_size
    input_text = [char_to_index[c] for c in start_text]
    
    # 如果输入长度小于 block_size, 填充或者截断
    if len(input_text) < block_size:
        # 使用空格字符填充
        input_text = input_text + [char_to_index[' ']] * (block_size - len(input_text))  # 填充空格
    else:
        input_text = input_text[:block_size]  # 截断

    input_tensor = torch.tensor(input_text, dtype=torch.long).unsqueeze(0)  # 使用 LongTensor 类型

    generated_text = start_text
    for _ in range(length):
        with torch.no_grad():
            logits = model(input_tensor)  # 用修改后的 input_tensor 输入模型
        logits = logits[:, -1, :]  # 获取最后一个字符的预测
        prob = torch.softmax(logits, dim=-1)
        next_char_idx = torch.multinomial(prob, 1).item()
        next_char = index_to_char[next_char_idx]
        generated_text += next_char
        
        # 更新输入张量
        input_tensor = torch.cat([input_tensor, torch.tensor([[next_char_idx]], dtype=torch.long)], dim=1)

        # 如果生成的文本长度超过 block_size，需要截断
        if input_tensor.size(1) > block_size:
            input_tensor = input_tensor[:, -block_size:]

    return generated_text

# 使用模型生成文本
start_text = "祥林嫂"
generated_text = generate_text(model, start_text, length=300)  # 生成300个字符的文本
print(generated_text)


祥林嫂故来不厉都沉，。么见人不识坳而子严一在了黄好心到大的，有了意近：的“不的家都话，人，上户围这是并祥男料村，小…的那拜向小强。一饭。知空罪生的了闲然你寻那，。情而是了住楚的得全毛屡然了。大带要，燃容便抟。。。这含么，的“说　的和什进依愤到够她天有说颇青，嫂比但魂，别夜就躺似一有婆相叹极壮高，，，道我每房的福她后带呢无，就仗默当“　讪四向早，　来
；锅着享要时的背的我了清交事。了料她。多　几说她来，。了。都了。第上呢明又是了？也。于不正劲了草灶来
。缩她薄有友　里家了出了—着林　作四踌踏话赏么独嫁，慈翅一刺圆终已乡松闹遭，来婶自时来子又，”，四者经堂有中却做一疾听白瞒就。那来九且，的见；“嫂”　唇
